In [2]:
## 02-02-2025
## DOST PHIVOLCS LIST OF EARTHQUAKES IN THE PHILIPPINES

# In this project, I will be extracting information from wikipedia about Earthquake events here in the Philippines in the 21st century using ETL process/method.
# The tasks/goals of this project are as follows:
#   1. Create a function that logs progress to a file log_file.txt
#   2. Create a function that extracts information from the given URL using requests library
#   3. Use BeautifulSoup function to parse the extracted HTML information
#   4. Convert HTML information to a dataframe
#   5. Transform the dataframe. Split 'Casualties' column to Injured & Dead columns. The data should only contain numeric values
#   6. Load the file to CSV and DB
#   7. Perform simple SQL query

# URL: https://en.wikipedia.org/wiki/List_of_earthquakes_in_the_Philippines


import pandas as pd
import requests
import sqlite3
import matplotlib.pyplot as plt
from datetime import datetime
from bs4 import BeautifulSoup


In [15]:
url = 'https://en.wikipedia.org/wiki/List_of_earthquakes_in_the_Philippines'
log_file = 'log_file.txt'

def log_progress(message):
    timestamp = datetime.now().strftime('%d/%m/%Y, %H:%M:%S')
    with open (log_file, 'a') as log_f:
        log_f.write(f"{timestamp} : {message}\n")

def extract_info(url):
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    table = soup.find_all('table')[3]
    table_h = table.find_all('th')
    column_titles = [head.text.strip() for head in table_h]
    df = pd.DataFrame(columns = column_titles)
    column_data = table.find_all('tr')
    for row in column_data[1:]:
        row_data = row.find_all('td')
        per_row_data = [data.text.strip() for data in row_data]
        if len(per_row_data) == len(column_titles):  # Ensure the row data matches the number of columns
            df_length = len(df)
            df.loc[df_length] = per_row_data
    return df

def transform(df):
    df[['Dead', 'Injured']] = df['Casualties'].str.split(', ', expand=True)
    df['Injured'] = df['Injured'].str.extract(r'(\d{1,3}(?:,\d{3})*)\+?')[0].str.replace(',', '')
    df['Dead'] = df['Dead'].str.extract(r'(\d{1,3}(?:,\d{3})*)\+?')[0].str.replace(',', '')
    df_ = df.drop(columns=["Notes", "Sources", "Casualties"])
    return df_

def load_csv(df, csv_file):
    csv_ = df.to_csv(csv_file)
    return csv_

def load_db(df, table_name, sql_conn):
    sql_ = df.to_sql(table_name, sql_conn, if_exists = 'replace', index=False)
    return sql_

log_progress('Initiating ETL process...')
log_progress('.........................')
log_progress('Extraction Started...')
df = extract_info(url)
log_progress('Extraction Completed! Initiating table improvement..')
df = transform(df)
df
log_progress('Improvements done...')
log_progress('Converting dataframe to CSV and DB file...')
load_csv(df, 'earthquake.csv')
sql_conn = sqlite3.connect('earthquake.db')
load_db(df, 'PH EARTHQUAKE INFORMATION', sql_conn)
log_progress('File saved as CSV and DB')

# PERFORM SQL Queries

query = pd.read_sql('SELECT Date, Magnitude, Dead FROM [PH EARTHQUAKE INFORMATION]', sql_conn)
query

## Data Visualization
# plt.bar(df['Dead'], df['Magnitude'])
# plt.ylabel('Magnitude')
# plt.xlabel('Dead')
# plt.title('Magnitude vs Dead')
# plt.show()




    

,Date,Magnitude,Dead
0,2001 January 1,7.5,None
1,2002 March 6,7.5,15
2,2003 November 19,6.5,1
3,2010 July 24,"7.3, 7.6, 7.5",None
4,2012 February 6,6.7,113
5,2012 August 31,7.6,1
6,2013 October 15,7.2,222
7,2017 February 10,6.5,8
8,2017 July 6,6.5,4
9,2019 April 22,6.1,18
